In [1]:
import os

In [2]:
%pwd

'c:\\Users\\admin\\OneDrive\\Dokumen\\Portofolio\\Data Sceince (Python)\\End-to-end\\research'

In [4]:
os.chdir('../')

In [5]:
%pwd

'c:\\Users\\admin\\OneDrive\\Dokumen\\Portofolio\\Data Sceince (Python)\\End-to-end'

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [11]:
from end_to_end_project.constants import *  # (CONFIG_FILE_PATH, PARAMS_FILE_PATH, SCHEMA_FILE_PATH)
from end_to_end_project.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH
    ):
    
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_url = config.source_url,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config

In [12]:
import os
import urllib.request as request
import zipfile
from end_to_end_project import logger
from end_to_end_project.utils.common import get_size

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_file
            )
            logger.info(f"Downloaded file: {filename} with following info: {headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path) 
            logger.info(f"Extracted zip file to : {unzip_path}")

In [ ]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e 

[2025-07-29 15:02:53,851: INFO: common: YAML file: config\config.yaml loaded successfully]
[2025-07-29 15:02:53,889: INFO: common: YAML file: params.yaml loaded successfully]
[2025-07-29 15:02:53,897: INFO: common: YAML file: schema.yaml loaded successfully]
[2025-07-29 15:02:53,899: INFO: common: Created directory at: artifacts]
[2025-07-29 15:02:53,901: INFO: common: Created directory at: artifacts/data_ingestion]
[2025-07-29 15:02:55,326: INFO: 760908179: Downloaded file: artifacts/data_ingestion/data.zip with following info: Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: C2B8:A6F19:7D83FD:8F970B:6888802B
Accept-Ranges: bytes
Date: Tue